In [2]:
import requests as r

import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt 
#plt.style.use('fivethirtyeight')

Import the original data file first.

Create a copy of the data file to work on.

In [4]:
#pd.set_option('display.max_columns', None)

df = pd.read_csv('dataset/steam.csv')

# work on this copy for cleaning data
cleaned_df = df.copy()

cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27075 entries, 0 to 27074
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   appid             27075 non-null  int64  
 1   name              27075 non-null  object 
 2   release_date      27075 non-null  object 
 3   english           27075 non-null  int64  
 4   developer         27074 non-null  object 
 5   publisher         27061 non-null  object 
 6   platforms         27075 non-null  object 
 7   required_age      27075 non-null  int64  
 8   categories        27075 non-null  object 
 9   genres            27075 non-null  object 
 10  steamspy_tags     27075 non-null  object 
 11  achievements      27075 non-null  int64  
 12  positive_ratings  27075 non-null  int64  
 13  negative_ratings  27075 non-null  int64  
 14  average_playtime  27075 non-null  int64  
 15  median_playtime   27075 non-null  int64  
 16  owners            27075 non-null  object

Drop columns not relevant.

In [21]:
# Delete rows where irrelavent
cleaned_df = cleaned_df.drop(columns=["developer", "publisher", "required_age", "platforms"])

cleaned_df.head()

,appid,name,release_date,english,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
0,10,Counter-Strike,2000-11-01,1,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,124534,3339,17612,317,10000000-20000000,7.19
1,20,Team Fortress Classic,1999-04-01,1,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,3318,633,277,62,5000000-10000000,3.99
2,30,Day of Defeat,2003-05-01,1,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,0,3416,398,187,34,5000000-10000000,3.99
3,40,Deathmatch Classic,2001-06-01,1,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,1273,267,258,184,5000000-10000000,3.99
4,50,Half-Life: Opposing Force,1999-11-01,1,Single-player;Multi-player;Valve Anti-Cheat en...,Action,FPS;Action;Sci-fi,0,5250,288,624,415,5000000-10000000,3.99


<insert description>

In [22]:
new_columns = []

for i in ["categories", "steamspy_tags", "genres"]:
    possible = cleaned_df[i].str.split(";").explode().unique()
    for p in possible:
        new_column = cleaned_df[i].str.contains(p, regex=False).astype(int)
        new_columns.append(pd.Series(new_column, name=i + "_" + p))

cleaned_df = pd.concat([cleaned_df] + new_columns, axis=1)
cleaned_df = cleaned_df.drop(columns=["categories", "steamspy_tags", "genres"])

cleaned_df.head()


,appid,name,release_date,english,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,...,genres_Web Publishing,genres_Education,genres_Software Training,genres_Sexual Content,genres_Audio Production,genres_Game Development,genres_Photo Editing,genres_Accounting,genres_Documentary,genres_Tutorial
0,10,Counter-Strike,2000-11-01,1,0,124534,3339,17612,317,10000000-20000000,...,0,0,0,0,0,0,0,0,0,0
1,20,Team Fortress Classic,1999-04-01,1,0,3318,633,277,62,5000000-10000000,...,0,0,0,0,0,0,0,0,0,0
2,30,Day of Defeat,2003-05-01,1,0,3416,398,187,34,5000000-10000000,...,0,0,0,0,0,0,0,0,0,0
3,40,Deathmatch Classic,2001-06-01,1,0,1273,267,258,184,5000000-10000000,...,0,0,0,0,0,0,0,0,0,0
4,50,Half-Life: Opposing Force,1999-11-01,1,0,5250,288,624,415,5000000-10000000,...,0,0,0,0,0,0,0,0,0,0


Drop non-English games. This make it easier to do natural language processing on the dataset later.

In [23]:
print("Dataset dimension before drop: ", cleaned_df.shape)

Dataset dimension before drop:  (27075, 408)


As you can see there are 27075 rows for both English and non-English games.

In [24]:
cleaned_df = cleaned_df[(cleaned_df["english"] == 1)]
print("Dataset dimension after remove non-English games: ", cleaned_df.shape)


Dataset dimension after remove non-English games:  (26564, 408)


As you can see there are 26564 rows for English games only.

Since now all games are English, can drop the english column because it is unnecessary.

In [25]:
cleaned_df = cleaned_df.drop(columns="english")

In [26]:
print("Dataset dimension after remove non-English games: ", cleaned_df.shape)

Dataset dimension after remove non-English games:  (26564, 407)


From 40 columns, now left 39 columns after removing english column. Also shown in below cleaned_df.info(), no more english column.

In [27]:
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26564 entries, 0 to 27074
Columns: 407 entries, appid to genres_Tutorial
dtypes: float64(1), int32(397), int64(6), object(3)
memory usage: 42.5+ MB


In [28]:
cleaned_df.head(n=1000)

,appid,name,release_date,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price,...,genres_Web Publishing,genres_Education,genres_Software Training,genres_Sexual Content,genres_Audio Production,genres_Game Development,genres_Photo Editing,genres_Accounting,genres_Documentary,genres_Tutorial
0,10,Counter-Strike,2000-11-01,0,124534,3339,17612,317,10000000-20000000,7.19,...,0,0,0,0,0,0,0,0,0,0
1,20,Team Fortress Classic,1999-04-01,0,3318,633,277,62,5000000-10000000,3.99,...,0,0,0,0,0,0,0,0,0,0
2,30,Day of Defeat,2003-05-01,0,3416,398,187,34,5000000-10000000,3.99,...,0,0,0,0,0,0,0,0,0,0
3,40,Deathmatch Classic,2001-06-01,0,1273,267,258,184,5000000-10000000,3.99,...,0,0,0,0,0,0,0,0,0,0
4,50,Half-Life: Opposing Force,1999-11-01,0,5250,288,624,415,5000000-10000000,3.99,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,65780,ARMA: Gold Edition,2011-07-13,0,548,209,22,22,500000-1000000,5.99,...,0,0,0,0,0,0,0,0,0,0
996,65790,ARMA: Cold War Assault,2011-08-08,0,3195,942,28,39,1000000-2000000,3.49,...,0,0,0,0,0,0,0,0,0,0
997,65800,Dungeon Defenders,2011-10-18,114,10693,932,1993,1205,1000000-2000000,9.99,...,0,0,0,0,0,0,0,0,0,0
998,65930,The Bureau: XCOM Declassified,2013-08-22,41,6661,2923,201,168,2000000-5000000,14.99,...,0,0,0,0,0,0,0,0,0,0


We use the rough estimate fr the "owners" category as we only have the range, we use the average 

In [29]:
owner = cleaned_df["owners"].str.split("-")
owner = owner.apply(lambda x: (int(x[0]) + int(x[1]))/2)
owner = owner.astype(int)
cleaned_df["owners"] = owner

Release date is updated to be measured in terms of days until 2024 Jan 1.
2024 Jan 1 has been arbitrarily chosen but it should not affect anything in the long run and is after all the dates in the dataset.

In [30]:
# Calculate the age column
cleaned_df["age"] = (pd.Timestamp("2024-01-01") - pd.to_datetime(cleaned_df["release_date"])).dt.days

# Drop the "release_date" column
cleaned_df = cleaned_df.drop(columns=["release_date"])

cleaned_df.head()

C:\Users\graph\AppData\Local\Temp\ipykernel_9408\2325596901.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  cleaned_df["age"] = (pd.Timestamp("2024-01-01") - pd.to_datetime(cleaned_df["release_date"])).dt.days


,appid,name,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price,categories_Multi-player,...,genres_Education,genres_Software Training,genres_Sexual Content,genres_Audio Production,genres_Game Development,genres_Photo Editing,genres_Accounting,genres_Documentary,genres_Tutorial,age
0,10,Counter-Strike,0,124534,3339,17612,317,15000000,7.19,1,...,0,0,0,0,0,0,0,0,0,8461
1,20,Team Fortress Classic,0,3318,633,277,62,7500000,3.99,1,...,0,0,0,0,0,0,0,0,0,9041
2,30,Day of Defeat,0,3416,398,187,34,7500000,3.99,1,...,0,0,0,0,0,0,0,0,0,7550
3,40,Deathmatch Classic,0,1273,267,258,184,7500000,3.99,1,...,0,0,0,0,0,0,0,0,0,8249
4,50,Half-Life: Opposing Force,0,5250,288,624,415,7500000,3.99,1,...,0,0,0,0,0,0,0,0,0,8827


In [31]:
# Filter rows based on the condition
cleaned_df = cleaned_df[(cleaned_df["positive_ratings"] + cleaned_df["negative_ratings"] >= 25)]

# Calculate positive ratio
cleaned_df["positive_ratio"] = cleaned_df["positive_ratings"] / (cleaned_df["positive_ratings"] + cleaned_df["negative_ratings"])

# Drop columns in one step
cleaned_df = cleaned_df.drop(columns=["positive_ratings", "negative_ratings"])
cleaned_df.head()

C:\Users\graph\AppData\Local\Temp\ipykernel_9408\2548901122.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  cleaned_df["positive_ratio"] = cleaned_df["positive_ratings"] / (cleaned_df["positive_ratings"] + cleaned_df["negative_ratings"])


,appid,name,achievements,average_playtime,median_playtime,owners,price,categories_Multi-player,categories_Online Multi-Player,categories_Local Multi-Player,...,genres_Software Training,genres_Sexual Content,genres_Audio Production,genres_Game Development,genres_Photo Editing,genres_Accounting,genres_Documentary,genres_Tutorial,age,positive_ratio
0,10,Counter-Strike,0,17612,317,15000000,7.19,1,1,1,...,0,0,0,0,0,0,0,0,8461,0.973888
1,20,Team Fortress Classic,0,277,62,7500000,3.99,1,1,1,...,0,0,0,0,0,0,0,0,9041,0.839787
2,30,Day of Defeat,0,187,34,7500000,3.99,1,0,0,...,0,0,0,0,0,0,0,0,7550,0.895648
3,40,Deathmatch Classic,0,258,184,7500000,3.99,1,1,1,...,0,0,0,0,0,0,0,0,8249,0.826623
4,50,Half-Life: Opposing Force,0,624,415,7500000,3.99,1,0,0,...,0,0,0,0,0,0,0,0,8827,0.947996


# Heres what the cleaned dataset looks like

In [32]:
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15288 entries, 0 to 27047
Columns: 406 entries, appid to positive_ratio
dtypes: float64(2), int32(398), int64(5), object(1)
memory usage: 24.3+ MB


In [33]:
cleaned_df.head()

,appid,name,achievements,average_playtime,median_playtime,owners,price,categories_Multi-player,categories_Online Multi-Player,categories_Local Multi-Player,...,genres_Software Training,genres_Sexual Content,genres_Audio Production,genres_Game Development,genres_Photo Editing,genres_Accounting,genres_Documentary,genres_Tutorial,age,positive_ratio
0,10,Counter-Strike,0,17612,317,15000000,7.19,1,1,1,...,0,0,0,0,0,0,0,0,8461,0.973888
1,20,Team Fortress Classic,0,277,62,7500000,3.99,1,1,1,...,0,0,0,0,0,0,0,0,9041,0.839787
2,30,Day of Defeat,0,187,34,7500000,3.99,1,0,0,...,0,0,0,0,0,0,0,0,7550,0.895648
3,40,Deathmatch Classic,0,258,184,7500000,3.99,1,1,1,...,0,0,0,0,0,0,0,0,8249,0.826623
4,50,Half-Life: Opposing Force,0,624,415,7500000,3.99,1,0,0,...,0,0,0,0,0,0,0,0,8827,0.947996


Export the cleaned dataset to a CSV file.

In [5]:
cleaned_df.to_csv('dataset/steam_cleaned.csv', index=False)